In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset
import random

In [2]:
# Set seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\Code\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [4]:
df = pd.read_excel(r"references\NEW_COMBINED_FINAL_Subject_info.xlsx")
df = df[df['iteration']=='old']

In [5]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [6]:
import torch
import torch.nn as nn

class VGG3D(nn.Module):
    def __init__(self, num_classes=1, input_shape=(1,160, 160, 96)):
        super(VGG3D, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels=input_shape[0], out_channels=8, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=8, out_channels=8, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(in_channels=8, out_channels=16, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=16, out_channels=16, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        )
        
        # This will be updated after running the model to find the correct flattened size
        self.fc1 = nn.Sequential(
            nn.Linear(38400, 128),  # This number will be updated
            nn.BatchNorm1d(128),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        if(num_classes==2):
            self.out = nn.Linear(64, 2)
            self.out_act = nn.Sigmoid()
        else:
            self.out = nn.Linear(64, num_classes)
            self.out_act = nn.Softmax(dim=1)

    def forward(self, x, drop_prob=0.8):
       # print("Input shape:", x.shape)
        x = self.conv1(x)
        #print("After Conv1:", x.shape)
        x = self.conv2(x)
       # print("After Conv2:", x.shape)
        x = self.conv3(x)
       # print("After Conv3:", x.shape)
        x = self.conv4(x)
       # print("After Conv4:", x.shape)
        x = x.view(x.size(0), -1)
        #print("Shape before FC1:", x.shape)
        x = self.fc1(x)
        x = nn.Dropout(drop_prob)(x)
        x = self.fc2(x)
        prob = self.out(x)
        return prob

In [7]:
def select_optimizer(model, config):
    """Select optimizer based on configuration."""
    lr = config.get('lr', 0.001)  # Default learning rate
    weight_decay = config.get('weight_decay', 0)  # Default weight decay
    
    if config['optimizer'] == 'Adam':
        return optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif config['optimizer'] == 'SGD':
        momentum = config.get('momentum', 0.9)  # Default momentum for SGD
        return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise ValueError("Unsupported optimizer")

def select_criterion(config):
    """Select loss criterion based on configuration."""
    if config['loss_criterion'] == 'Cross-Entropy':
        return nn.CrossEntropyLoss()
    elif config['loss_criterion'] == 'BCEWithLogits':
        return nn.BCEWithLogitsLoss()
    else:
        raise ValueError("Unsupported loss criterion")
    
    

In [19]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim






def run_experiment(df, config):
    train_dataset, val_dataset, test_dataset = load_datasets(df, image_type = config['image_type'], sample_size = config['sample_size'], loss_type = config['loss_criterion'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    device = torch.device("cuda")
    model = VGG3D().to(device)
    
    criterion = select_criterion(config)
    optimizer = select_optimizer(model, config)


    # Training and validation
    train_accuracies, val_accuracies, val_losses, results_df = train_and_validate(model, train_loader, val_loader, criterion, optimizer, config['num_epochs'], config['patience'], device, config['loss_criterion'])
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device, loss_type = config['loss_criterion'])
    
    # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    report_filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    onnx_filename = os.path.join('models', f'{formatted_time}_Model.')
          # Save the model to ONNX


    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(report_filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')
        results_df.to_excel(writer, sheet_name='Training_Results')

# Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    dummy_input = torch.randn(4, 1, 160, 160, 96, device=device)  # Adjust size according to your model's input
    torch.onnx.export(model, dummy_input, onnx_filename, export_params=True)

    return report_filename, train_accuracies[-1], val_accuracies[-1], test_accuracy


# Example configuration and use case
config = {
    'optimizer': 'SGD',
    'lr' : 0.001,
    'loss_criterion': 'Cross-Entropy',
    'num_epochs': 100,
    'batch_size': 8,
    'patience': 30,
    'Description' : 'TEST WITH PET DIMENSIONS - Higher learning rate',
    'image_type' : 'Resampled Images_fused',
    'weight_decay' : 0,
    'momentum' : 0.7,
    'sample_size' : None

}

# Single experiment

In [20]:
  #column_mapping = {
  #          'Co-registered PET': 'Co-registered PET',
   #         'Fused Images': 'Fused Images',
    #        'Masked PET': 'Masked PET',
     #       'Spatial Normalization': 'Spatial Normalization',
      #      'Resampled Images(Co-registered PET)': 'Resampled Images(Co-registered PET)',
       #     'Resampled Images(Masked PET)': 'Resampled Images(Masked PET)',
        #    'Resampled Images(Spatial Normalization)': 'Resampled Images(Spatial Normalization)',
         #   'Resampled Images_fused': 'Resampled Images_fused'
        #}

image_types = ['Fused Images (RESAMPLED TO PET)', 'Original PET (OLD)']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)



Working on image type: Fused Images (RESAMPLED TO PET)


Running Experiments:   0%|          | 0/24 [2:54:29<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# MANY experiments

In [15]:
def run_multiple_experiments(df):
    """Run experiments for all combinations of configurations."""
    optimizers = ['SGD', 'Adam']
    loss_criterions = ['BCEWithLogits']
    learning_rates = [0.001, 0.05, 0.01, 0.1]
    image_types = [
        'Fused Images (RESAMPLED TO PET)']

    num_epochs = 200
    patience = 30
    batch_size = 8

    results = []
    total = len(optimizers) * len(loss_criterions) * len(learning_rates) * len(image_types)
    progress_bar = tqdm(total=total, desc='Running Experiments', leave=True)

    for lr in learning_rates:    
        for optimizer in optimizers:
            for loss_criterion in loss_criterions:
                    for image_type in image_types:
                        config = {
                            'optimizer': optimizer,
                            'loss_criterion': loss_criterion,
                            'lr': lr,
                            'num_epochs': num_epochs,
                            'batch_size': batch_size,
                            'patience': patience,
                            'image_type': image_type,
                            'Description': 'MANY EXPERIEMTN TESTs',
                            'sample_size' : None,
                            'model' : 'VGG3D(ORIGINAL DIMENSIONS)'
                        }
                        
                    report_filename, train_acc, val_acc, test_acc = run_experiment(df, config)

                    results.append({
                        'Optimizer': optimizer,
                        'Loss Criterion': loss_criterion,
                        'Learning Rate': lr,
                        'Image Type': image_type,
                        'Report File': report_filename,
                        'Train Accuracy': train_acc,
                        'Validation Accuracy': val_acc,
                        'Test Accuracy': test_acc
                    })
                    progress_bar.update(1)

    progress_bar.close()

    # Convert results to DataFrame and save or return
    results_df = pd.DataFrame(results)
    results_df.to_csv('experiment_results_summary.csv', index=False)
    return results_df

In [16]:
results_df = run_multiple_experiments(df)











Epoch 1/200 - Train: 100%|██████████| 9/9 [00:05<00:00,  1.52it/s]


Epoch 1: Train Loss: 0.6721 - Train Accuracy: 58.33%




Epoch 1/200 - Validate: 100%|██████████| 1/1 [00:00<00:00,  2.73it/s]


Epoch 1: Validation Loss: 0.6987 - Validation Accuracy: 50.00%












Epoch 2/200 - Train: 100%|██████████| 9/9 [00:04<00:00,  2.12it/s]


...











Epoch 33/200 - Train: 100%|██████████| 9/9 [00:03<00:00,  2.48it/s]


Epoch 33: Train Loss: 0.7057 - Train Accuracy: 50.00%




Epoch 33/200 - Validate: 100%|██████████| 1/1 [00:00<00:00,  5.71it/s]


Epoch 33: Validation Loss: 0.8047 - Validation Accuracy: 25.00%
Early stopping triggered after 33 epochs due to no improvement in validation loss or accuracy.






Testing: 100%|██████████| 3/3 [00:00<00:00,  5.80it/s]
C:\Users\Micha\AppData\Local\Temp\ipykernel_23624\1549752429.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(
Running Experiments: 100%|██████████| 8/8 [24:44<00:00, 185.58s/it]
